<a href="https://colab.research.google.com/github/NUROISEA/anime-webui-colab/blob/main/elysium_anime_unified_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>🌟 <code>Elysium Anime Colab</code></h2>

- 🤔 [How to use](https://github.com/NUROISEA/anime-webui-colab#-how-to-use)

In [ ]:
#@title 🚀 `Launch web UI`

#@markdown 🧐 Select what Elysium model you want to use
model_version = "V3" #@param ["V1", "V2", "V3"]

#@markdown Click the play button on the left to install the web UI and it's dependencies

#@markdown It takes around 3-5 minutes before you can see a `gradio.app` link to the web UI

try:
  has_run
except NameError:
  has_run = False

web_ui_folder = "/content/stable-diffusion-webui"
extensions_folder = f"{web_ui_folder}/extensions"
models_folder = f"{web_ui_folder}/models/Stable-diffusion"
vae_folder = f"{web_ui_folder}/models/VAE"
embeddings_folder = f"{web_ui_folder}/embeddings"

ui_config_dictionary = {
  "txt2img/Negative prompt/value": "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry",
  "txt2img/Sampling method/value": "DPM++ 2M Karras"
}

model_dictionary = {
  "V1":"https://huggingface.co/hesw23168/SD-Elysium-Model/resolve/main/Elysium_Anime_V1.ckpt",
  "V2":"https://huggingface.co/hesw23168/SD-Elysium-Model/resolve/main/Elysium_Anime_V2.ckpt",
  "V3":"https://huggingface.co/hesw23168/SD-Elysium-Model/resolve/main/Elysium_Anime_V3.safetensors",
}

vae_link = "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt"
vae_name = vae_link.split("/")[-1]

model_link = model_dictionary[model_version]
model_name = model_link.split("/")[-1]

git_flags = "-q"
wget_flags = "-q --show-progress"

if not has_run:
  has_run = True

  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl

  !git clone {git_flags} https://github.com/NUROISEA/stable-diffusion-webui
  !git clone {git_flags} https://github.com/yfszzx/stable-diffusion-webui-images-browser {extensions_folder}/stable-diffusion-webui-images-browser
  !git clone {git_flags} https://github.com/DominikDoom/a1111-sd-webui-tagcomplete {extensions_folder}/a1111-sd-webui-tagcomplete

  !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/ui-config.json -O {web_ui_folder}/ui-config.json 
  !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/config.json -O {web_ui_folder}/config.json
  !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/styles.csv -O {web_ui_folder}/styles.csv

  !wget {wget_flags} https://huggingface.co/nick-x-hacker/bad-artist/resolve/main/bad-artist.pt -O {embeddings_folder}/bad-artist.pt
  !wget {wget_flags} https://huggingface.co/datasets/Nerfgun3/bad_prompt/resolve/main/bad_prompt_version2.pt -O {embeddings_folder}/bad_prompt_version2.pt
  
  !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/utility.py -O /content/utility.py
  import utility
  utility.dictionary_to_json(f"{web_ui_folder}/ui-config.json", ui_config_dictionary)
   
  !wget {wget_flags} {vae_link} -O {vae_folder}/{vae_name}

try:
  models_downloaded
except NameError:
  models_downloaded = []

if model_version not in models_downloaded:
  models_downloaded.append(model_version)
  !wget {wget_flags} {model_link} -O {models_folder}/{model_name}

arg_list = [
  "--share --xformers",
  f"--ckpt {models_folder}/{model_name}",
  f"--vae-path {vae_folder}/{vae_name}"
]

%cd {web_ui_folder}
args = " ".join(arg_list)
!python launch.py {args}

In [ ]:
#@title 💾 `Saving images`

#@markdown The zip file will be visible at the files tab.

#@markdown ----

from datetime import datetime
str_date = datetime.today().strftime('%Y-%m-%d-%H%M%S')
archive_name = f"outputs-{str_date}.zip"

%cd /content/stable-diffusion-webui
print("Zipping...")
!zip -qr /content/{archive_name} outputs
print(f"\033[92mZipped. You can now find {archive_name} at the files tab.\033[0m")

# ----

#@markdown 📲 Copy zip to Google Drive? (Recommended)
copy_to_gdrive = False #@param {type:'boolean'}
#@markdown 📂 The folder path to copy the zip file, if the above is checked, if left blank it will be in `AI/Generated`
gdrive_folder = "AI/Generated" #@param { 'type': 'string' }

if copy_to_gdrive:
  from google.colab import drive

  print("Mounting to Google Drive...")
  drive.mount('/content/drive')
  if gdrive_folder == "":
    gdrive_folder = "AI/Generated"
  
  drive_folder = f"/content/drive/MyDrive/{gdrive_folder}"
  
  !mkdir -p {drive_folder}
  !cp /content/{archive_name} {drive_folder}
  print(f"\033[92mCopied to {gdrive_folder}!\033[0m")

#@markdown ----

#@markdown 📤 Upload zip to a temporary file hosting? 
upload_to_temp = False #@param {type:'boolean'}

#@markdown 🌐 The website/host to upload to. 

#@markdown You can read more about the individual site's limits [here](https://github.com/NUROISEA/anime-webui-colab/blob/main/docs/temp_file_hosts.md). 
temp_file_host = "transfer.sh" #@param ["bashupload.com", "keep.sh", "temp.sh", "transfer.sh"]

host_dictionary = {
  "bashupload.com": "https://bashupload.com",
  "keep.sh": "https://free.keep.sh",
  "temp.sh": "https://temp.sh",
  "transfer.sh": f"https://transfer.sh/{archive_name}",
}
host_link = host_dictionary[temp_file_host] # not sure if colab can do this directly at {}

if upload_to_temp:
  print(f"Uploading to {temp_file_host}...")
  !curl -T /content/{archive_name} {host_link}  
  print("\n\033[92m^^^ This is your download link! ^^^\033[0m")

#@markdown ----

#@markdown ❌ Delete the files in the colab? (this wont delete the saved zip)
delete_local_files = False #@param {type:'boolean'}
if delete_local_files:
  %cd /content/stable-diffusion-webui
  !rm -rf outputs
  print("\033[91mDeleted the outputs folder.\033[0m")